In [ ]:
#@title Environment Setup
import torch
if torch.cuda.is_available() == True:
  "CUDA device name:", torch.cuda.get_device_name(0)
  !nvidia-smi
else:
  import sys
  print("No CUDA GPU found. Did you forget to use the correct runtime type?")
  sys.exit()

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
INSTALL_COMFYUI_MANAGER = False  #@param {type:"boolean"}
INSTALL_ANIMATEDIFF = False  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = False  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['INSTALL_COMFYUI_MANAGER'] = INSTALL_COMFYUI_MANAGER
OPTIONS['INSTALL_ANIMATEDIFF'] = INSTALL_ANIMATEDIFF
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

if OPTIONS['USE_GOOGLE_DRIVE']:
  print("Mounting Google Drive...")
  %cd /

  from google.colab import drive
  drive.mount('/content/drive')

  AI_WORKSPACE = "/content/drive/MyDrive/temp"
  COMFY_WORKSPACE = "/content/drive/MyDrive/temp/ComfyUI"
else:
  current_dir = !pwd
  AI_WORKSPACE = f"{current_dir[0]}/temp"
  COMFY_WORKSPACE = f"{current_dir[0]}/temp/ComfyUI"

import os
print("Creating AI workspace (if it doesn't exist)")
os.makedirs(AI_WORKSPACE, exist_ok=True)

%cd $AI_WORKSPACE

![ ! -d $COMFY_WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $COMFY_WORKSPACE
   
!nvidia-smi

if OPTIONS['UPDATE_COMFY_UI']:
  print("-= Updating ComfyUI =-")
  !git pull

print("-= Install dependencies =-")
# that download link might be getting out of date version
%pip install https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117
%pip install onnxruntime-gpu color-matcher simpleeval
%pip install --use-pep517 facexlib
%pip install onnxruntime-gpu==1.15.1
%pip install insightface onnxruntime

import onnxruntime as ort

available_providers = ort.get_available_providers()
print("Available providers:", available_providers)

if "CUDAExecutionProvider" in available_providers:
    print("CUDAExecutionProvider is available. You're good to go!")
else:
    import sys
    print("CUDAExecutionProvider is NOT available. Check your installation.")
    sys.exit()

if OPTIONS['INSTALL_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

if OPTIONS['INSTALL_ANIMATEDIFF']:
  %cd ../
  ![ ! -d ComfyUI-AnimateDiff-Evolved ] && echo -= Initial setup AnimateDiff =- && git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved
  %cd ComfyUI-AnimateDiff-Evolved
  !git pull

%cd $COMFY_WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !pwd
  print("-= Install custom nodes dependencies =-")
  ![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py"
# might need to add other custom node dependencies here. i remember i had to manually download some
